In [1]:
"""
========================================================================
HOTEL REVENUE ANALYTICS - MONTE CARLO SIMULATION
========================================================================
Author: Berke Ilgun
Method: Monte Carlo Simulation for Overbooking Optimization

Purpose:
- Optimize overbooking strategy for BOTH hotel types
- City Hotel vs Resort Hotel separate analysis
- Compare strategies and identify best approach for each
- Quantify risk and expected outcomes

Skills: Monte Carlo simulation, Risk analysis, Decision modeling
========================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting setup
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)

print("=" * 70)
print("🎲 MONTE CARLO SIMULATION - OVERBOOKING OPTIMIZATION")
print("=" * 70)
print("\n🎯 Goal: Optimize overbooking for City Hotel AND Resort Hotel")
print("✅ Libraries loaded successfully!")

🎲 MONTE CARLO SIMULATION - OVERBOOKING OPTIMIZATION

🎯 Goal: Optimize overbooking for City Hotel AND Resort Hotel
✅ Libraries loaded successfully!


In [2]:
# ========================================================================
# THE BUSINESS PROBLEM
# ========================================================================
print("\n" + "=" * 70)
print("📋 THE OVERBOOKING CHALLENGE")
print("=" * 70)

print("""
🏨 THE DILEMMA:

Hotels must balance two competing priorities:

EMPTY ROOMS = LOST REVENUE
- Some guests cancel or don't show up
- Empty rooms = permanent revenue loss
- Can't recover that night's revenue

OVERBOOKING = GUEST RELOCATION COST
- If everyone shows up when overbooked
- Must relocate guests to another hotel
- Expensive + customer dissatisfaction

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎲 WHY MONTE CARLO SIMULATION?

Problem: Reality has VARIABILITY
- No-shows aren't constant (varies 10-20%)
- Some nights: many cancellations
- Some nights: everyone shows up
- This randomness creates risk

Solution: Monte Carlo Simulation
- Simulate 10,000 different nights
- Account for natural variation
- See full distribution of outcomes
- Make data-driven decisions with confidence

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🏨 WHY ANALYZE BOTH HOTELS SEPARATELY?

City Hotel vs Resort Hotel are DIFFERENT:
- Different sizes (capacity)
- Different cancellation patterns
- Different room rates
- Different customer types

→ They need DIFFERENT overbooking strategies!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")


📋 THE OVERBOOKING CHALLENGE

🏨 THE DILEMMA:

Hotels must balance two competing priorities:

EMPTY ROOMS = LOST REVENUE
- Some guests cancel or don't show up
- Empty rooms = permanent revenue loss
- Can't recover that night's revenue

OVERBOOKING = GUEST RELOCATION COST
- If everyone shows up when overbooked
- Must relocate guests to another hotel
- Expensive + customer dissatisfaction

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎲 WHY MONTE CARLO SIMULATION?

Problem: Reality has VARIABILITY
- No-shows aren't constant (varies 10-20%)
- Some nights: many cancellations
- Some nights: everyone shows up
- This randomness creates risk

Solution: Monte Carlo Simulation
- Simulate 10,000 different nights
- Account for natural variation
- See full distribution of outcomes
- Make data-driven decisions with confidence

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🏨 WHY ANALYZE BOTH HOTELS SEPARATELY?

City Hotel vs Resort Hotel are DIFFERENT:
-

In [3]:
# ========================================================================
# STEP 1: LOAD & ANALYZE BOTH HOTEL TYPES
# ========================================================================
print("\n" + "=" * 70)
print("📥 STEP 1: Loading and analyzing both hotel types")
print("=" * 70)

# Load the cleaned CSV file
print("\n📂 Loading hotel data...")
df_hotels = pd.read_csv('../data/hotel_bookings_cleaned.csv')
print("✅ Data loaded!")

# Analyze BOTH hotels
print("\n🏨 HOTELS IN DATASET:")
hotel_types = df_hotels['hotel'].value_counts()
print(hotel_types)

# Store comprehensive stats for BOTH hotels
all_hotels_data = {}

for hotel_name in df_hotels['hotel'].unique():
    
    print("\n" + "=" * 70)
    print(f"🏨 ANALYZING: {hotel_name.upper()}")
    print("=" * 70)
    
    hotel_data = df_hotels[df_hotels['hotel'] == hotel_name]
    
    # Basic statistics
    total_bookings = len(hotel_data)
    cancellations = hotel_data['is_canceled'].sum()
    completed = total_bookings - cancellations
    no_show_rate = cancellations / total_bookings
    
    # Financial data (completed bookings only)
    completed_bookings = hotel_data[hotel_data['is_canceled'] == 0]
    avg_adr = completed_bookings['adr'].mean()
    median_adr = completed_bookings['adr'].median()
    
    print(f"\n📊 BOOKING STATISTICS:")
    print(f"   Total Bookings: {total_bookings:,}")
    print(f"   Completed: {completed:,} ({(completed/total_bookings)*100:.1f}%)")
    print(f"   Cancelled: {cancellations:,} ({no_show_rate*100:.1f}%)")
    
    print(f"\n💰 PRICING:")
    print(f"   Average ADR: €{avg_adr:.2f}")
    print(f"   Median ADR: €{median_adr:.2f}")
    
    # Estimate capacity from daily bookings
    daily_bookings = hotel_data.groupby('arrival_date').size()
    p90 = daily_bookings.quantile(0.90)
    p95 = daily_bookings.quantile(0.95)
    max_daily = daily_bookings.max()
    
    # Use 95th percentile as capacity estimate
    estimated_capacity = int(p95)
    
    print(f"\n📅 DAILY PATTERNS:")
    print(f"   Average: {daily_bookings.mean():.1f} bookings/day")
    print(f"   90th percentile: {p90:.0f}")
    print(f"   95th percentile: {p95:.0f}")
    print(f"   Maximum: {max_daily:.0f}")
    
    print(f"\n🏨 ESTIMATED CAPACITY: ~{estimated_capacity} rooms")
    print(f"   (Based on 95th percentile)")
    
    # Calculate relocation cost (industry standard: 1.5-1.6x ADR)
    relocation_cost = avg_adr * 1.6
    
    print(f"\n💵 RELOCATION COST ESTIMATE:")
    print(f"   Industry standard: 1.5-1.6x ADR")
    print(f"   Calculated: €{relocation_cost:.2f}")
    print(f"   (Alternate room + transport + goodwill)")
    
    # Store all data
    all_hotels_data[hotel_name] = {
        'total_bookings': total_bookings,
        'completed': completed,
        'cancellations': cancellations,
        'no_show_rate': no_show_rate,
        'avg_adr': avg_adr,
        'median_adr': median_adr,
        'estimated_capacity': estimated_capacity,
        'relocation_cost': relocation_cost,
        'p90_daily': int(p90),
        'p95_daily': int(p95),
        'max_daily': int(max_daily)
    }

print("\n" + "=" * 70)
print("✅ BOTH HOTELS ANALYZED!")
print("=" * 70)

# Quick comparison
print("\n📊 QUICK COMPARISON:")

hotel_names_list = list(all_hotels_data.keys())
for metric in ['estimated_capacity', 'no_show_rate', 'avg_adr']:
    h1_val = all_hotels_data[hotel_names_list[0]][metric]
    h2_val = all_hotels_data[hotel_names_list[1]][metric]
    
    if metric == 'no_show_rate':
        print(f"\n{metric.replace('_', ' ').title()}:")
        print(f"   {hotel_names_list[0]}: {h1_val:.1%}")
        print(f"   {hotel_names_list[1]}: {h2_val:.1%}")
    elif metric == 'avg_adr':
        print(f"\n{metric.replace('_', ' ').title()}:")
        print(f"   {hotel_names_list[0]}: €{h1_val:.2f}")
        print(f"   {hotel_names_list[1]}: €{h2_val:.2f}")
    else:
        print(f"\n{metric.replace('_', ' ').title()}:")
        print(f"   {hotel_names_list[0]}: {h1_val}")
        print(f"   {hotel_names_list[1]}: {h2_val}")


📥 STEP 1: Loading and analyzing both hotel types

📂 Loading hotel data...
✅ Data loaded!

🏨 HOTELS IN DATASET:
hotel
City Hotel      65646
Resort Hotel    31745
Name: count, dtype: int64

🏨 ANALYZING: RESORT HOTEL

📊 BOOKING STATISTICS:
   Total Bookings: 31,745
   Completed: 22,761 (71.7%)
   Cancelled: 8,984 (28.3%)

💰 PRICING:
   Average ADR: €91.07
   Median ADR: €71.60

📅 DAILY PATTERNS:
   Average: 52.1 bookings/day
   90th percentile: 79
   95th percentile: 96
   Maximum: 175

🏨 ESTIMATED CAPACITY: ~96 rooms
   (Based on 95th percentile)

💵 RELOCATION COST ESTIMATE:
   Industry standard: 1.5-1.6x ADR
   Calculated: €145.72
   (Alternate room + transport + goodwill)

🏨 ANALYZING: CITY HOTEL

📊 BOOKING STATISTICS:
   Total Bookings: 65,646
   Completed: 38,549 (58.7%)
   Cancelled: 27,097 (41.3%)

💰 PRICING:
   Average ADR: €109.30
   Median ADR: €104.00

📅 DAILY PATTERNS:
   Average: 107.8 bookings/day
   90th percentile: 170
   95th percentile: 192
   Maximum: 322

🏨 ESTIMATED 

In [4]:
# ========================================================================
# STEP 2: DEFINE MONTE CARLO SIMULATION FUNCTION
# ========================================================================
print("\n" + "=" * 70)
print("⚙️  STEP 2: Defining simulation function")
print("=" * 70)

def run_monte_carlo_simulation(
    hotel_name,
    total_rooms,
    avg_room_rate,
    no_show_rate,
    relocation_cost,
    num_simulations=10000,
    overbooking_levels=range(0, 26, 2)
):
    """
    Run Monte Carlo simulation for hotel overbooking strategy
    
    Parameters:
    -----------
    hotel_name : str
        Name of the hotel
    total_rooms : int
        Hotel capacity
    avg_room_rate : float
        Average daily rate
    no_show_rate : float
        Historical cancellation/no-show rate
    relocation_cost : float
        Cost to relocate one guest
    num_simulations : int
        Number of Monte Carlo iterations
    overbooking_levels : range/list
        Overbooking percentages to test
    
    Returns:
    --------
    results_df : DataFrame
        Simulation results for all overbooking levels
    """
    
    print(f"\n🎲 Running Monte Carlo for: {hotel_name}")
    print(f"   Capacity: {total_rooms} rooms")
    print(f"   ADR: €{avg_room_rate:.2f}")
    print(f"   No-show rate: {no_show_rate:.1%}")
    print(f"   Simulations: {num_simulations:,}")
    
    results = []
    
    for overbook_pct in overbooking_levels:
        
        # Calculate bookings to accept
        bookings_accepted = int(total_rooms * (1 + overbook_pct/100))
        
        # Arrays to store results
        revenues = []
        relocations = []
        occupied = []
        
        # Run simulations
        for sim in range(num_simulations):
            
            # Simulate actual arrivals (binomial distribution)
            # Each booking has probability (1 - no_show_rate) of showing up
            actual_arrivals = np.random.binomial(
                n=bookings_accepted,
                p=(1 - no_show_rate)
            )
            
            # Calculate outcomes
            if actual_arrivals <= total_rooms:
                # No problem - all guests accommodated
                revenue = actual_arrivals * avg_room_rate
                guests_relocated = 0
                rooms_occupied = actual_arrivals
            else:
                # Overbooking - must relocate excess
                guests_relocated = actual_arrivals - total_rooms
                rooms_occupied = total_rooms
                
                # Revenue = rooms sold - relocation costs
                revenue = (total_rooms * avg_room_rate) - \
                         (guests_relocated * relocation_cost)
            
            revenues.append(revenue)
            relocations.append(guests_relocated)
            occupied.append(rooms_occupied)
        
        # Calculate statistics
        results.append({
            'overbook_pct': overbook_pct,
            'bookings_accepted': bookings_accepted,
            'avg_revenue': np.mean(revenues),
            'median_revenue': np.median(revenues),
            'std_revenue': np.std(revenues),
            'revenue_5th': np.percentile(revenues, 5),
            'revenue_95th': np.percentile(revenues, 95),
            'avg_relocations': np.mean(relocations),
            'prob_overbook': np.mean([r > 0 for r in relocations]),
            'avg_occupancy': np.mean(occupied),
            'occupancy_rate': (np.mean(occupied) / total_rooms) * 100
        })
    
    results_df = pd.DataFrame(results)
    
    print(f"✅ Simulation complete for {hotel_name}!")
    
    return results_df

print("✅ Simulation function defined!")


⚙️  STEP 2: Defining simulation function
✅ Simulation function defined!


In [5]:
# ========================================================================
# STEP 2: DEFINE MONTE CARLO SIMULATION FUNCTION
# ========================================================================
print("\n" + "=" * 70)
print("⚙️  STEP 2: Defining simulation function")
print("=" * 70)

def run_monte_carlo_simulation(
    hotel_name,
    total_rooms,
    avg_room_rate,
    no_show_rate,
    relocation_cost,
    num_simulations=10000,
    overbooking_levels=range(0, 26, 2)
):
    """
    Run Monte Carlo simulation for hotel overbooking strategy
    
    Parameters:
    -----------
    hotel_name : str
        Name of the hotel
    total_rooms : int
        Hotel capacity
    avg_room_rate : float
        Average daily rate
    no_show_rate : float
        Historical cancellation/no-show rate
    relocation_cost : float
        Cost to relocate one guest
    num_simulations : int
        Number of Monte Carlo iterations
    overbooking_levels : range/list
        Overbooking percentages to test
    
    Returns:
    --------
    results_df : DataFrame
        Simulation results for all overbooking levels
    """
    
    print(f"\n🎲 Running Monte Carlo for: {hotel_name}")
    print(f"   Capacity: {total_rooms} rooms")
    print(f"   ADR: €{avg_room_rate:.2f}")
    print(f"   No-show rate: {no_show_rate:.1%}")
    print(f"   Simulations: {num_simulations:,}")
    
    results = []
    
    for overbook_pct in overbooking_levels:
        
        # Calculate bookings to accept
        bookings_accepted = int(total_rooms * (1 + overbook_pct/100))
        
        # Arrays to store results
        revenues = []
        relocations = []
        occupied = []
        
        # Run simulations
        for sim in range(num_simulations):
            
            # Simulate actual arrivals (binomial distribution)
            # Each booking has probability (1 - no_show_rate) of showing up
            actual_arrivals = np.random.binomial(
                n=bookings_accepted,
                p=(1 - no_show_rate)
            )
            
            # Calculate outcomes
            if actual_arrivals <= total_rooms:
                # No problem - all guests accommodated
                revenue = actual_arrivals * avg_room_rate
                guests_relocated = 0
                rooms_occupied = actual_arrivals
            else:
                # Overbooking - must relocate excess
                guests_relocated = actual_arrivals - total_rooms
                rooms_occupied = total_rooms
                
                # Revenue = rooms sold - relocation costs
                revenue = (total_rooms * avg_room_rate) - \
                         (guests_relocated * relocation_cost)
            
            revenues.append(revenue)
            relocations.append(guests_relocated)
            occupied.append(rooms_occupied)
        
        # Calculate statistics
        results.append({
            'overbook_pct': overbook_pct,
            'bookings_accepted': bookings_accepted,
            'avg_revenue': np.mean(revenues),
            'median_revenue': np.median(revenues),
            'std_revenue': np.std(revenues),
            'revenue_5th': np.percentile(revenues, 5),
            'revenue_95th': np.percentile(revenues, 95),
            'avg_relocations': np.mean(relocations),
            'prob_overbook': np.mean([r > 0 for r in relocations]),
            'avg_occupancy': np.mean(occupied),
            'occupancy_rate': (np.mean(occupied) / total_rooms) * 100
        })
    
    results_df = pd.DataFrame(results)
    
    print(f"✅ Simulation complete for {hotel_name}!")
    
    return results_df

print("✅ Simulation function defined!")


⚙️  STEP 2: Defining simulation function
✅ Simulation function defined!


In [6]:
# ========================================================================
# STEP 3: RUN MONTE CARLO SIMULATIONS FOR BOTH HOTELS
# ========================================================================
print("\n" + "=" * 70)
print("🎲 STEP 3: Running simulations for BOTH hotels")
print("=" * 70)

import time

# Simulation settings
NUM_SIMULATIONS = 10000
OVERBOOKING_LEVELS = list(range(0, 26, 2))  # 0%, 2%, 4%, ..., 24%

print(f"\n⚙️  SIMULATION SETTINGS:")
print(f"   Iterations per strategy: {NUM_SIMULATIONS:,}")
print(f"   Strategies to test: {len(OVERBOOKING_LEVELS)} (from 0% to 24%)")
print(f"   Total scenarios: {NUM_SIMULATIONS * len(OVERBOOKING_LEVELS) * 2:,} (both hotels)")

# Store results for both hotels
simulation_results = {}

start_time = time.time()

# Run simulation for EACH hotel
for hotel_name, hotel_data in all_hotels_data.items():
    
    print("\n" + "=" * 70)
    print(f"🏨 HOTEL: {hotel_name.upper()}")
    print("=" * 70)
    
    # Extract parameters
    rooms = hotel_data['estimated_capacity']
    adr = hotel_data['avg_adr']
    no_show = hotel_data['no_show_rate']
    relocation = hotel_data['relocation_cost']
    
    print(f"\n📋 Parameters:")
    print(f"   Rooms: {rooms}")
    print(f"   ADR: €{adr:.2f}")
    print(f"   No-show Rate: {no_show:.2%}")
    print(f"   Relocation Cost: €{relocation:.2f}")
    
    # Run Monte Carlo simulation
    results_df = run_monte_carlo_simulation(
        hotel_name=hotel_name,
        total_rooms=rooms,
        avg_room_rate=adr,
        no_show_rate=no_show,
        relocation_cost=relocation,
        num_simulations=NUM_SIMULATIONS,
        overbooking_levels=OVERBOOKING_LEVELS
    )
    
    # Find optimal strategy
    optimal_idx = results_df['avg_revenue'].idxmax()
    optimal = results_df.iloc[optimal_idx]
    baseline = results_df[results_df['overbook_pct'] == 0].iloc[0]
    
    # Calculate gains
    revenue_gain = optimal['avg_revenue'] - baseline['avg_revenue']
    revenue_gain_pct = (revenue_gain / baseline['avg_revenue']) * 100
    
    print(f"\n🎯 OPTIMAL STRATEGY:")
    print(f"   Overbooking Level: {optimal['overbook_pct']:.0f}%")
    print(f"   Bookings to Accept: {optimal['bookings_accepted']:.0f} (for {rooms} rooms)")
    print(f"   Revenue: €{optimal['avg_revenue']:,.2f}/night")
    print(f"   Gain: €{revenue_gain:,.2f}/night ({revenue_gain_pct:+.1f}%)")
    print(f"   Risk: {optimal['prob_overbook']*100:.1f}% overbooking probability")
    
    # Store everything
    simulation_results[hotel_name] = {
        'results_df': results_df,
        'optimal': optimal,
        'baseline': baseline,
        'revenue_gain': revenue_gain,
        'revenue_gain_pct': revenue_gain_pct,
        'parameters': {
            'rooms': rooms,
            'adr': adr,
            'no_show': no_show,
            'relocation': relocation
        }
    }

elapsed = time.time() - start_time

print("\n" + "=" * 70)
print("✅ ALL SIMULATIONS COMPLETE!")
print("=" * 70)
print(f"\n⏱️  Total computation time: {elapsed:.1f} seconds")
print(f"📊 Total scenarios analyzed: {NUM_SIMULATIONS * len(OVERBOOKING_LEVELS) * 2:,}")

# Quick summary
print("\n📋 SUMMARY:")
for hotel_name, data in simulation_results.items():
    print(f"\n🏨 {hotel_name}:")
    print(f"   Optimal: {data['optimal']['overbook_pct']:.0f}% overbooking")
    print(f"   Revenue gain: €{data['revenue_gain']:,.2f}/night ({data['revenue_gain_pct']:.1f}%)")
    print(f"   Annual gain: €{data['revenue_gain'] * 365:,.2f}")


🎲 STEP 3: Running simulations for BOTH hotels

⚙️  SIMULATION SETTINGS:
   Iterations per strategy: 10,000
   Strategies to test: 13 (from 0% to 24%)
   Total scenarios: 260,000 (both hotels)

🏨 HOTEL: RESORT HOTEL

📋 Parameters:
   Rooms: 96
   ADR: €91.07
   No-show Rate: 28.30%
   Relocation Cost: €145.72

🎲 Running Monte Carlo for: Resort Hotel
   Capacity: 96 rooms
   ADR: €91.07
   No-show rate: 28.3%
   Simulations: 10,000
✅ Simulation complete for Resort Hotel!

🎯 OPTIMAL STRATEGY:
   Overbooking Level: 24%
   Bookings to Accept: 119 (for 96 rooms)
   Revenue: €7,770.16/night
   Gain: €1,493.10/night (+23.8%)
   Risk: 1.2% overbooking probability

🏨 HOTEL: CITY HOTEL

📋 Parameters:
   Rooms: 191
   ADR: €109.30
   No-show Rate: 41.28%
   Relocation Cost: €174.87

🎲 Running Monte Carlo for: City Hotel
   Capacity: 191 rooms
   ADR: €109.30
   No-show rate: 41.3%
   Simulations: 10,000
✅ Simulation complete for City Hotel!

🎯 OPTIMAL STRATEGY:
   Overbooking Level: 24%
   Bookin

In [7]:
# ========================================================================
# STEP 4: DETAILED COMPARISON ANALYSIS
# ========================================================================
print("\n" + "=" * 70)
print("📊 STEP 4: Comparing both hotels")
print("=" * 70)

# Get hotel names
hotel_names = list(simulation_results.keys())
hotel1_name = hotel_names[0]
hotel2_name = hotel_names[1]

# Get data for each hotel
hotel1 = simulation_results[hotel1_name]
hotel2 = simulation_results[hotel2_name]

# Create comprehensive comparison table
comparison_data = {
    'Metric': [
        '═══ HOTEL CHARACTERISTICS ═══',
        'Hotel Capacity (rooms)',
        'Average ADR',
        'Historical No-show Rate',
        'Relocation Cost',
        '',
        '═══ OPTIMAL STRATEGY ═══',
        'Recommended Overbooking %',
        'Bookings to Accept',
        'Extra Bookings Above Capacity',
        '',
        '═══ REVENUE IMPACT ═══',
        'Baseline Revenue (no overbook)',
        'Optimal Revenue (with overbook)',
        'Revenue Gain per Night',
        'Revenue Gain Percentage',
        'Monthly Revenue Gain',
        'Annual Revenue Gain',
        '',
        '═══ RISK METRICS ═══',
        'Overbooking Probability',
        'Expected Incidents per Year',
        'Avg Guests Relocated per Incident',
        'Occupancy Rate',
        '',
        '═══ CONFIDENCE INTERVALS ═══',
        'Revenue 5th Percentile',
        'Revenue 95th Percentile',
        'Revenue Range (90% CI)'
    ],
    hotel1_name: [
        '',
        hotel1['parameters']['rooms'],
        f"€{hotel1['parameters']['adr']:.2f}",
        f"{hotel1['parameters']['no_show']:.1%}",
        f"€{hotel1['parameters']['relocation']:.2f}",
        '',
        '',
        f"{hotel1['optimal']['overbook_pct']:.0f}%",
        f"{hotel1['optimal']['bookings_accepted']:.0f}",
        f"{hotel1['optimal']['bookings_accepted'] - hotel1['parameters']['rooms']:.0f}",
        '',
        '',
        f"€{hotel1['baseline']['avg_revenue']:,.2f}",
        f"€{hotel1['optimal']['avg_revenue']:,.2f}",
        f"€{hotel1['revenue_gain']:,.2f}",
        f"{hotel1['revenue_gain_pct']:.2f}%",
        f"€{hotel1['revenue_gain'] * 30:,.2f}",
        f"€{hotel1['revenue_gain'] * 365:,.2f}",
        '',
        '',
        f"{hotel1['optimal']['prob_overbook']*100:.1f}%",
        f"{hotel1['optimal']['prob_overbook']*365:.0f}",
        f"{hotel1['optimal']['avg_relocations']:.2f}",
        f"{hotel1['optimal']['occupancy_rate']:.1f}%",
        '',
        '',
        f"€{hotel1['optimal']['revenue_5th']:,.2f}",
        f"€{hotel1['optimal']['revenue_95th']:,.2f}",
        f"€{hotel1['optimal']['revenue_95th'] - hotel1['optimal']['revenue_5th']:,.2f}"
    ],
    hotel2_name: [
        '',
        hotel2['parameters']['rooms'],
        f"€{hotel2['parameters']['adr']:.2f}",
        f"{hotel2['parameters']['no_show']:.1%}",
        f"€{hotel2['parameters']['relocation']:.2f}",
        '',
        '',
        f"{hotel2['optimal']['overbook_pct']:.0f}%",
        f"{hotel2['optimal']['bookings_accepted']:.0f}",
        f"{hotel2['optimal']['bookings_accepted'] - hotel2['parameters']['rooms']:.0f}",
        '',
        '',
        f"€{hotel2['baseline']['avg_revenue']:,.2f}",
        f"€{hotel2['optimal']['avg_revenue']:,.2f}",
        f"€{hotel2['revenue_gain']:,.2f}",
        f"{hotel2['revenue_gain_pct']:.2f}%",
        f"€{hotel2['revenue_gain'] * 30:,.2f}",
        f"€{hotel2['revenue_gain'] * 365:,.2f}",
        '',
        '',
        f"{hotel2['optimal']['prob_overbook']*100:.1f}%",
        f"{hotel2['optimal']['prob_overbook']*365:.0f}",
        f"{hotel2['optimal']['avg_relocations']:.2f}",
        f"{hotel2['optimal']['occupancy_rate']:.1f}%",
        '',
        '',
        f"€{hotel2['optimal']['revenue_5th']:,.2f}",
        f"€{hotel2['optimal']['revenue_95th']:,.2f}",
        f"€{hotel2['optimal']['revenue_95th'] - hotel2['optimal']['revenue_5th']:,.2f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("\n📋 COMPREHENSIVE COMPARISON TABLE:")
print("=" * 70)
print(comparison_df.to_string(index=False))

# Calculate combined impact
total_annual_gain = (hotel1['revenue_gain'] * 365) + (hotel2['revenue_gain'] * 365)

print("\n" + "=" * 70)
print("💰 COMBINED PORTFOLIO IMPACT")
print("=" * 70)
print(f"\n   {hotel1_name} Annual Gain: €{hotel1['revenue_gain'] * 365:,.2f}")
print(f"   {hotel2_name} Annual Gain: €{hotel2['revenue_gain'] * 365:,.2f}")
print(f"   ─────────────────────────────────────────")
print(f"   TOTAL ANNUAL GAIN: €{total_annual_gain:,.2f}")

# Key insights
print("\n" + "=" * 70)
print("💡 KEY INSIGHTS & DIFFERENCES")
print("=" * 70)

# Compare optimal strategies
if hotel1['optimal']['overbook_pct'] > hotel2['optimal']['overbook_pct']:
    more_aggressive = hotel1_name
    less_aggressive = hotel2_name
    diff = hotel1['optimal']['overbook_pct'] - hotel2['optimal']['overbook_pct']
else:
    more_aggressive = hotel2_name
    less_aggressive = hotel1_name
    diff = hotel2['optimal']['overbook_pct'] - hotel1['optimal']['overbook_pct']

print(f"\n1️⃣  STRATEGY DIFFERENCES:")
print(f"   • {more_aggressive}: {simulation_results[more_aggressive]['optimal']['overbook_pct']:.0f}% overbooking (MORE aggressive)")
print(f"   • {less_aggressive}: {simulation_results[less_aggressive]['optimal']['overbook_pct']:.0f}% overbooking (MORE conservative)")
print(f"   • Difference: {diff:.0f} percentage points")

# Explain why
if simulation_results[more_aggressive]['parameters']['no_show'] > simulation_results[less_aggressive]['parameters']['no_show']:
    print(f"\n   Why? {more_aggressive} has higher no-show rate:")
    print(f"   • {more_aggressive}: {simulation_results[more_aggressive]['parameters']['no_show']:.1%}")
    print(f"   • {less_aggressive}: {simulation_results[less_aggressive]['parameters']['no_show']:.1%}")
    print(f"   → Higher no-shows = more room for overbooking")

# Compare revenue opportunities
if hotel1['revenue_gain'] * 365 > hotel2['revenue_gain'] * 365:
    bigger_opp = hotel1_name
    smaller_opp = hotel2_name
else:
    bigger_opp = hotel2_name
    smaller_opp = hotel1_name

print(f"\n2️⃣  REVENUE OPPORTUNITY:")
print(f"   • {bigger_opp}: €{simulation_results[bigger_opp]['revenue_gain'] * 365:,.2f} annual gain (LARGER)")
print(f"   • {smaller_opp}: €{simulation_results[smaller_opp]['revenue_gain'] * 365:,.2f} annual gain")
print(f"   • {bigger_opp} has {(simulation_results[bigger_opp]['revenue_gain'] * 365) / (simulation_results[smaller_opp]['revenue_gain'] * 365):.1f}x larger opportunity")

# Compare risks
print(f"\n3️⃣  RISK COMPARISON:")
print(f"   • {hotel1_name}: {hotel1['optimal']['prob_overbook']*100:.1f}% incidents (~{hotel1['optimal']['prob_overbook']*365:.0f}/year)")
print(f"   • {hotel2_name}: {hotel2['optimal']['prob_overbook']*100:.1f}% incidents (~{hotel2['optimal']['prob_overbook']*365:.0f}/year)")

if hotel1['optimal']['prob_overbook'] > hotel2['optimal']['prob_overbook']:
    higher_risk = hotel1_name
    lower_risk = hotel2_name
else:
    higher_risk = hotel2_name
    lower_risk = hotel1_name

print(f"   • {higher_risk} has higher risk but justified by:")
print(f"     - Higher no-show rate")
print(f"     - Greater revenue upside")

# Compare sizes
print(f"\n4️⃣  HOTEL SIZE IMPACT:")
print(f"   • {hotel1_name}: {hotel1['parameters']['rooms']} rooms")
print(f"   • {hotel2_name}: {hotel2['parameters']['rooms']} rooms")

if hotel1['parameters']['rooms'] > hotel2['parameters']['rooms']:
    larger = hotel1_name
    smaller = hotel2_name
else:
    larger = hotel2_name
    smaller = hotel1_name

print(f"   • {larger} is larger → more absolute revenue impact")
print(f"   • {smaller} may have higher % gains")

print("\n✅ Comparison analysis complete!")


📊 STEP 4: Comparing both hotels

📋 COMPREHENSIVE COMPARISON TABLE:
                           Metric Resort Hotel    City Hotel
    ═══ HOTEL CHARACTERISTICS ═══                           
           Hotel Capacity (rooms)           96           191
                      Average ADR       €91.07       €109.30
          Historical No-show Rate        28.3%         41.3%
                  Relocation Cost      €145.72       €174.87
                                                            
         ═══ OPTIMAL STRATEGY ═══                           
        Recommended Overbooking %          24%           24%
               Bookings to Accept          119           236
    Extra Bookings Above Capacity           23            45
                                                            
           ═══ REVENUE IMPACT ═══                           
   Baseline Revenue (no overbook)    €6,277.06    €12,263.59
  Optimal Revenue (with overbook)    €7,770.16    €15,130.69
           Revenu

In [8]:
# ========================================================================
# DIAGNOSTIC: WHY ARE BOTH 24%?
# ========================================================================
print("\n" + "=" * 70)
print("🔍 DIAGNOSTIC: Investigating 24% result")
print("=" * 70)

# Check both hotels' full results
for hotel_name, data in simulation_results.items():
    
    print(f"\n🏨 {hotel_name}:")
    print("=" * 70)
    
    results = data['results_df']
    
    # Show last 5 overbooking levels
    print("\n📊 Revenue by overbooking level (last 5):")
    print(results[['overbook_pct', 'avg_revenue']].tail(5).to_string(index=False))
    
    # Check if revenue is STILL INCREASING at 24%
    revenue_20 = results[results['overbook_pct'] == 20]['avg_revenue'].values[0]
    revenue_22 = results[results['overbook_pct'] == 22]['avg_revenue'].values[0]
    revenue_24 = results[results['overbook_pct'] == 24]['avg_revenue'].values[0]
    
    print(f"\n📈 Revenue trend at high levels:")
    print(f"   20%: €{revenue_20:,.2f}")
    print(f"   22%: €{revenue_22:,.2f}")
    print(f"   24%: €{revenue_24:,.2f}")
    
    # Check if it's still growing
    growth_20_22 = revenue_22 - revenue_20
    growth_22_24 = revenue_24 - revenue_22
    
    print(f"\n   Growth 20→22%: €{growth_20_22:,.2f}")
    print(f"   Growth 22→24%: €{growth_22_24:,.2f}")
    
    if growth_22_24 > 100:  # Still growing significantly
        print(f"\n   ⚠️  WARNING: Revenue still increasing!")
        print(f"   → Optimal might be ABOVE 24%")
        print(f"   → We hit our testing limit")
    elif growth_22_24 < 10:
        print(f"\n   ✅ Revenue plateauing - 24% is likely optimal")
    else:
        print(f"\n   🤔 Moderate growth - 24% might be optimal")
    
    # Check risk levels
    prob_24 = results[results['overbook_pct'] == 24]['prob_overbook'].values[0]
    print(f"\n⚠️  Risk at 24%:")
    print(f"   Overbooking probability: {prob_24*100:.1f}%")
    print(f"   Expected incidents/year: {prob_24*365:.0f}")
    
    if prob_24 > 0.4:
        print(f"   🔴 HIGH RISK - Over 40% overbooking rate!")
    elif prob_24 > 0.3:
        print(f"   🟡 MODERATE-HIGH RISK")
    else:
        print(f"   🟢 ACCEPTABLE RISK")

print("\n" + "=" * 70)
print("💡 DIAGNOSIS:")
print("=" * 70)

# Check if both are truly optimal or hitting limits
both_at_24 = all(data['optimal']['overbook_pct'] == 24 for data in simulation_results.values())

if both_at_24:
    print("""
🔍 Both hotels show 24% as optimal.

This could mean:

1️⃣  HITTING TEST LIMIT (Most Likely):
   • We only tested up to 24%
   • Optimal might actually be 26-30%
   • Revenue is still increasing at 24%
   
2️⃣  GENUINELY OPTIMAL:
   • Both hotels have high no-show rates
   • 24% balances revenue vs risk
   • Revenue growth is slowing at this level
   
3️⃣  SIMULATION ISSUE:
   • Parameters might be off
   • Need to verify no-show rates
   
RECOMMENDATION:
→ Extend test range to 30% and re-run
→ Check if revenue still growing at 24%
""")
else:
    print("✅ Hotels have different optimal strategies - results are valid!")


🔍 DIAGNOSTIC: Investigating 24% result

🏨 Resort Hotel:

📊 Revenue by overbooking level (last 5):
 overbook_pct  avg_revenue
           16  7242.378985
           18  7380.091682
           20  7506.433055
           22  7645.717666
           24  7770.161085

📈 Revenue trend at high levels:
   20%: €7,506.43
   22%: €7,645.72
   24%: €7,770.16

   Growth 20→22%: €139.28
   Growth 22→24%: €124.44

   ⚠️  WARNING: Revenue still increasing!
   → Optimal might be ABOVE 24%
   → We hit our testing limit

⚠️  Risk at 24%:
   Overbooking probability: 1.2%
   Expected incidents/year: 4
   🟢 ACCEPTABLE RISK

🏨 City Hotel:

📊 Revenue by overbooking level (last 5):
 overbook_pct  avg_revenue
           16 14181.284531
           18 14445.319928
           20 14682.752867
           22 14950.930553
           24 15130.688404

📈 Revenue trend at high levels:
   20%: €14,682.75
   22%: €14,950.93
   24%: €15,130.69

   Growth 20→22%: €268.18
   Growth 22→24%: €179.76

   ⚠️  WARNING: Revenue still

In [9]:
# ========================================================================
# STEP 3 (REVISED): EXTENDED RANGE SIMULATION
# ========================================================================
print("\n" + "=" * 70)
print("🎲 RE-RUNNING WITH EXTENDED RANGE (0-40%)")
print("=" * 70)

print("\n💡 Why extending?")
print("   • Revenue still increasing at 24%")
print("   • Risk levels are acceptable")
print("   • Need to find TRUE optimal")

# NEW Extended range
NUM_SIMULATIONS = 10000
OVERBOOKING_LEVELS_EXTENDED = list(range(0, 42, 2))  # 0% to 40%

print(f"\n⚙️  NEW SIMULATION SETTINGS:")
print(f"   Iterations: {NUM_SIMULATIONS:,}")
print(f"   Range: 0% to 40% (was 0-24%)")
print(f"   Strategies: {len(OVERBOOKING_LEVELS_EXTENDED)}")

# Re-run for both hotels
simulation_results = {}

start_time = time.time()

for hotel_name, hotel_data in all_hotels_data.items():
    
    print("\n" + "=" * 70)
    print(f"🏨 RE-ANALYZING: {hotel_name.upper()}")
    print("=" * 70)
    
    # Extract parameters
    rooms = hotel_data['estimated_capacity']
    adr = hotel_data['avg_adr']
    no_show = hotel_data['no_show_rate']
    relocation = hotel_data['relocation_cost']
    
    print(f"\n📋 Parameters:")
    print(f"   Rooms: {rooms}")
    print(f"   ADR: €{adr:.2f}")
    print(f"   No-show: {no_show:.2%}")
    print(f"   Testing: 0-40% overbooking")
    
    # Run extended simulation
    results_df = run_monte_carlo_simulation(
        hotel_name=hotel_name,
        total_rooms=rooms,
        avg_room_rate=adr,
        no_show_rate=no_show,
        relocation_cost=relocation,
        num_simulations=NUM_SIMULATIONS,
        overbooking_levels=OVERBOOKING_LEVELS_EXTENDED
    )
    
    # Find optimal
    optimal_idx = results_df['avg_revenue'].idxmax()
    optimal = results_df.iloc[optimal_idx]
    baseline = results_df[results_df['overbook_pct'] == 0].iloc[0]
    
    revenue_gain = optimal['avg_revenue'] - baseline['avg_revenue']
    revenue_gain_pct = (revenue_gain / baseline['avg_revenue']) * 100
    
    print(f"\n🎯 OPTIMAL STRATEGY (UPDATED):")
    print(f"   Overbooking: {optimal['overbook_pct']:.0f}%")
    print(f"   Bookings: {optimal['bookings_accepted']:.0f} (capacity: {rooms})")
    print(f"   Revenue: €{optimal['avg_revenue']:,.2f}/night")
    print(f"   Gain: €{revenue_gain:,.2f}/night ({revenue_gain_pct:+.1f}%)")
    print(f"   Risk: {optimal['prob_overbook']*100:.1f}% overbooking probability")
    print(f"   Annual incidents: ~{optimal['prob_overbook']*365:.0f}")
    
    # Check if still at limit
    if optimal['overbook_pct'] == 40:
        print(f"\n   ⚠️  Still at upper limit! May need even more!")
    else:
        print(f"\n   ✅ Found true optimal (not at limit)")
    
    # Show revenue curve near optimal
    print(f"\n📊 Revenue around optimal:")
    nearby = results_df[
        (results_df['overbook_pct'] >= optimal['overbook_pct'] - 4) &
        (results_df['overbook_pct'] <= optimal['overbook_pct'] + 4)
    ][['overbook_pct', 'avg_revenue', 'prob_overbook']]
    print(nearby.to_string(index=False))
    
    # Store results
    simulation_results[hotel_name] = {
        'results_df': results_df,
        'optimal': optimal,
        'baseline': baseline,
        'revenue_gain': revenue_gain,
        'revenue_gain_pct': revenue_gain_pct,
        'parameters': {
            'rooms': rooms,
            'adr': adr,
            'no_show': no_show,
            'relocation': relocation
        }
    }

elapsed = time.time() - start_time

print("\n" + "=" * 70)
print("✅ EXTENDED SIMULATIONS COMPLETE!")
print("=" * 70)
print(f"\n⏱️  Time: {elapsed:.1f} seconds")

# Compare strategies
print("\n📊 FINAL OPTIMAL STRATEGIES:")
print("=" * 70)

for hotel_name, data in simulation_results.items():
    annual_gain = data['revenue_gain'] * 365
    print(f"\n🏨 {hotel_name}:")
    print(f"   Optimal: {data['optimal']['overbook_pct']:.0f}% overbooking")
    print(f"   Bookings: {data['optimal']['bookings_accepted']:.0f}")
    print(f"   Daily gain: €{data['revenue_gain']:,.2f} ({data['revenue_gain_pct']:.1f}%)")
    print(f"   Annual gain: €{annual_gain:,.2f}")
    print(f"   Risk: {data['optimal']['prob_overbook']*100:.1f}% ({data['optimal']['prob_overbook']*365:.0f} incidents/year)")
    
    # Risk assessment
    risk = data['optimal']['prob_overbook']
    if risk < 0.10:
        risk_level = "🟢 LOW"
    elif risk < 0.25:
        risk_level = "🟡 MODERATE"
    elif risk < 0.40:
        risk_level = "🟠 HIGH"
    else:
        risk_level = "🔴 VERY HIGH"
    
    print(f"   Risk Level: {risk_level}")

# Check if they're different now
if simulation_results[hotel_names[0]]['optimal']['overbook_pct'] != \
   simulation_results[hotel_names[1]]['optimal']['overbook_pct']:
    print(f"\n✅ Hotels have DIFFERENT optimal strategies!")
else:
    print(f"\n⚠️  Hotels still have same optimal strategy")
    print(f"   This suggests similar overbooking dynamics despite different characteristics")


🎲 RE-RUNNING WITH EXTENDED RANGE (0-40%)

💡 Why extending?
   • Revenue still increasing at 24%
   • Risk levels are acceptable
   • Need to find TRUE optimal

⚙️  NEW SIMULATION SETTINGS:
   Iterations: 10,000
   Range: 0% to 40% (was 0-24%)
   Strategies: 21

🏨 RE-ANALYZING: RESORT HOTEL

📋 Parameters:
   Rooms: 96
   ADR: €91.07
   No-show: 28.30%
   Testing: 0-40% overbooking

🎲 Running Monte Carlo for: Resort Hotel
   Capacity: 96 rooms
   ADR: €91.07
   No-show rate: 28.3%
   Simulations: 10,000
✅ Simulation complete for Resort Hotel!

🎯 OPTIMAL STRATEGY (UPDATED):
   Overbooking: 38%
   Bookings: 132 (capacity: 96)
   Revenue: €8,283.53/night
   Gain: €2,018.28/night (+32.2%)
   Risk: 36.1% overbooking probability
   Annual incidents: ~132

   ✅ Found true optimal (not at limit)

📊 Revenue around optimal:
 overbook_pct  avg_revenue  prob_overbook
           34  8222.214499         0.1791
           36  8274.120415         0.2687
           38  8283.530036         0.3612
       

In [11]:
# ========================================================================
# DEEP DIAGNOSTIC: What's Really Happening?
# ========================================================================
print("\n" + "=" * 70)
print("🔍 DEEP DIVE: Investigating unusual results")
print("=" * 70)

for hotel_name, data in simulation_results.items():
    
    print(f"\n🏨 {hotel_name}")
    print("=" * 70)
    
    params = data['parameters']
    results = data['results_df']
    
    # Show key parameters
    print(f"\n📊 Key Parameters:")
    print(f"   Capacity: {params['rooms']} rooms")
    print(f"   No-show rate: {params['no_show']:.2%}")
    print(f"   ADR: €{params['adr']:.2f}")
    
    # Calculate what no-show rate means in practice
    print(f"\n🎲 What No-Show Rate Means:")
    bookings_at_optimal = data['optimal']['bookings_accepted']
    expected_arrivals = bookings_at_optimal * (1 - params['no_show'])
    expected_no_shows = bookings_at_optimal * params['no_show']
    
    print(f"   At optimal ({data['optimal']['overbook_pct']:.0f}% overbooking):")
    print(f"   • Bookings accepted: {bookings_at_optimal:.0f}")
    print(f"   • Expected arrivals: {expected_arrivals:.1f}")
    print(f"   • Expected no-shows: {expected_no_shows:.1f}")
    print(f"   • Hotel capacity: {params['rooms']}")
    print(f"   • Expected overbooking: {max(0, expected_arrivals - params['rooms']):.1f} guests")
    
    # Show revenue trend at high levels
    print(f"\n📈 Revenue Trend (High Overbooking Levels):")
    high_levels = results[results['overbook_pct'] >= 30][['overbook_pct', 'avg_revenue', 'prob_overbook']]
    print(high_levels.to_string(index=False))
    
    # Check if revenue is still increasing
    rev_38 = results[results['overbook_pct'] == 38]['avg_revenue'].values[0]
    rev_40 = results[results['overbook_pct'] == 40]['avg_revenue'].values[0]
    growth = rev_40 - rev_38
    
    print(f"\n   Revenue growth 38→40%: €{growth:,.2f}")
    
    if growth > 100:
        print(f"   ⚠️  Still growing significantly!")
    else:
        print(f"   ✅ Growth slowing - likely near optimal")
    
    # Risk analysis
    print(f"\n⚠️  Risk Analysis at Optimal:")
    print(f"   Overbooking probability: {data['optimal']['prob_overbook']*100:.1f}%")
    
    if data['optimal']['prob_overbook'] < 0.01:
        print(f"   🤔 SUSPICIOUS: Less than 1% risk at {data['optimal']['overbook_pct']:.0f}% overbooking?")
        print(f"   → This suggests no-show rate is VERY high")
        print(f"   → OR simulation might have an issue")

# Check the no-show rates
print("\n" + "=" * 70)
print("🔍 NO-SHOW RATE ANALYSIS")
print("=" * 70)

for hotel_name, hotel_data in all_hotels_data.items():
    print(f"\n{hotel_name}:")
    print(f"   Cancellation rate: {hotel_data['no_show_rate']:.2%}")
    print(f"   Completed bookings: {hotel_data['completed']:,}")
    print(f"   Cancelled bookings: {hotel_data['cancellations']:,}")
    
    # What this means
    if hotel_data['no_show_rate'] > 0.30:
        print(f"   💡 HIGH cancellation rate (>{30}%)")
        print(f"   → Justifies aggressive overbooking")
    
    # Calculate safe overbooking
    safe_overbook = hotel_data['no_show_rate'] * 100
    print(f"\n   Conservative estimate:")
    print(f"   → Can safely overbook by ~{safe_overbook:.0f}%")
    print(f"   → Simulation suggests: {simulation_results[hotel_name]['optimal']['overbook_pct']:.0f}%")


🔍 DEEP DIVE: Investigating unusual results

🏨 Resort Hotel

📊 Key Parameters:
   Capacity: 96 rooms
   No-show rate: 28.30%
   ADR: €91.07

🎲 What No-Show Rate Means:
   At optimal (38% overbooking):
   • Bookings accepted: 132
   • Expected arrivals: 94.6
   • Expected no-shows: 37.4
   • Hotel capacity: 96
   • Expected overbooking: 0.0 guests

📈 Revenue Trend (High Overbooking Levels):
 overbook_pct  avg_revenue  prob_overbook
           30  8056.544335         0.0635
           32  8153.518438         0.1094
           34  8222.214499         0.1791
           36  8274.120415         0.2687
           38  8283.530036         0.3612
           40  8250.459754         0.4790

   Revenue growth 38→40%: €-33.07
   ✅ Growth slowing - likely near optimal

⚠️  Risk Analysis at Optimal:
   Overbooking probability: 36.1%

🏨 City Hotel

📊 Key Parameters:
   Capacity: 191 rooms
   No-show rate: 41.28%
   ADR: €109.30

🎲 What No-Show Rate Means:
   At optimal (40% overbooking):
   • Bookings 

In [15]:
# ========================================================================
# STEP 5: BUSINESS-REALISTIC RECOMMENDATIONS (BULLETPROOF)
# ========================================================================
print("\n" + "=" * 70)
print("💼 BUSINESS-REALISTIC OVERBOOKING STRATEGY")
print("=" * 70)

# Check what levels we have
print("\n🔍 Available overbooking levels in simulation:")
for hotel_name, data in simulation_results.items():
    levels = sorted(data['results_df']['overbook_pct'].unique())
    print(f"   {hotel_name}: {levels}")

print("\n" + "=" * 70)
print("📊 SCENARIO ANALYSIS: Multiple Risk Tolerance Levels")
print("=" * 70)

# Define scenarios using levels we KNOW exist (every 2% from 0-40)
scenario_levels = [10, 16, 20, 24, 30]  # Conservative, Moderate, Aggressive, Very Aggressive, Extreme

for hotel_name, data in simulation_results.items():
    
    print(f"\n🏨 {hotel_name.upper()}")
    print("=" * 70)
    
    results = data['results_df']
    baseline_rev = data['baseline']['avg_revenue']
    params = data['parameters']
    optimal = data['optimal']
    
    print(f"\n📊 Characteristics:")
    print(f"   Capacity: {params['rooms']} rooms")
    print(f"   Cancellation Rate: {params['no_show']:.1%}")
    print(f"   ADR: €{params['adr']:.2f}")
    
    print(f"\n📈 Overbooking Strategy Options:")
    print("=" * 70)
    
    scenario_data = []
    
    # Add scenarios at fixed levels
    for level in scenario_levels:
        # Find this level in results
        level_data = results[results['overbook_pct'] == level]
        
        if len(level_data) > 0:
            strategy = level_data.iloc[0]
            daily_gain = strategy['avg_revenue'] - baseline_rev
            annual_gain = daily_gain * 365
            
            # Determine scenario name
            if level <= 10:
                name = "Conservative"
            elif level <= 16:
                name = "Moderate"
            elif level <= 20:
                name = "Aggressive"
            elif level <= 24:
                name = "Very Aggressive"
            else:
                name = "Extreme"
            
            scenario_data.append({
                'Strategy': name,
                'Overbook': f"{level}%",
                'Bookings': f"{strategy['bookings_accepted']:.0f}",
                'Daily Gain': f"€{daily_gain:,.0f}",
                'Annual Gain': f"€{annual_gain:,.0f}",
                'Risk': f"{strategy['prob_overbook']*100:.1f}%",
                'Incidents/Yr': f"{strategy['prob_overbook']*365:.0f}"
            })
    
    # Add optimal
    opt_daily_gain = optimal['avg_revenue'] - baseline_rev
    opt_annual_gain = opt_daily_gain * 365
    
    scenario_data.append({
        'Strategy': f"Math Optimal",
        'Overbook': f"{optimal['overbook_pct']:.0f}%",
        'Bookings': f"{optimal['bookings_accepted']:.0f}",
        'Daily Gain': f"€{opt_daily_gain:,.0f}",
        'Annual Gain': f"€{opt_annual_gain:,.0f}",
        'Risk': f"{optimal['prob_overbook']*100:.1f}%",
        'Incidents/Yr': f"{optimal['prob_overbook']*365:.0f}"
    })
    
    scenario_df = pd.DataFrame(scenario_data)
    print(scenario_df.to_string(index=False))
    
    # Analysis
    if params['no_show'] > 0.35:
        print(f"\n   💡 INSIGHT: Very high cancellation rate ({params['no_show']:.0%})")
        print(f"      → Can tolerate aggressive overbooking")
        print(f"      → Even 30%+ overbooking has acceptable risk")

# Recommendations
print("\n" + "=" * 70)
print("💡 RECOMMENDED IMPLEMENTATION ROADMAP")
print("=" * 70)

for hotel_name, data in simulation_results.items():
    
    print(f"\n🏨 {hotel_name}:")
    print("-" * 70)
    
    results = data['results_df']
    params = data['parameters']
    baseline_rev = data['baseline']['avg_revenue']
    
    # Get key strategies (guaranteed to exist)
    phase1 = results[results['overbook_pct'] == 10].iloc[0]
    phase2 = results[results['overbook_pct'] == 16].iloc[0]  
    phase3 = results[results['overbook_pct'] == 20].iloc[0]
    optimal = data['optimal']
    
    # Phase 1
    p1_gain = (phase1['avg_revenue'] - baseline_rev) * 365
    print(f"\n   ⏸️  PHASE 1: Start Conservative (10%)")
    print(f"      Months 1-3")
    print(f"      Bookings: {phase1['bookings_accepted']:.0f} | Gain: €{p1_gain:,.0f}/year")
    print(f"      Risk: {phase1['prob_overbook']*100:.1f}% | Incidents: ~{phase1['prob_overbook']*365:.0f}/year")
    
    # Phase 2
    p2_gain = (phase2['avg_revenue'] - baseline_rev) * 365
    print(f"\n   ▶️  PHASE 2: Scale to Moderate (16%)")
    print(f"      Months 4-6")
    print(f"      Bookings: {phase2['bookings_accepted']:.0f} | Gain: €{p2_gain:,.0f}/year")
    print(f"      Risk: {phase2['prob_overbook']*100:.1f}% | Incidents: ~{phase2['prob_overbook']*365:.0f}/year")
    
    # Phase 3
    p3_gain = (phase3['avg_revenue'] - baseline_rev) * 365
    print(f"\n   ⏩ PHASE 3: Optimize to Aggressive (20%)")
    print(f"      Months 7-12")
    print(f"      Bookings: {phase3['bookings_accepted']:.0f} | Gain: €{p3_gain:,.0f}/year")
    print(f"      Risk: {phase3['prob_overbook']*100:.1f}% | Incidents: ~{phase3['prob_overbook']*365:.0f}/year")
    
    # Optimal consideration
    opt_gain = (optimal['avg_revenue'] - baseline_rev) * 365
    print(f"\n   🎯 POTENTIAL: Mathematical Optimal ({optimal['overbook_pct']:.0f}%)")
    print(f"      After 12+ months if successful")
    print(f"      Bookings: {optimal['bookings_accepted']:.0f} | Gain: €{opt_gain:,.0f}/year")
    print(f"      Risk: {optimal['prob_overbook']*100:.1f}% | Incidents: ~{optimal['prob_overbook']*365:.0f}/year")
    
    if optimal['prob_overbook'] < 0.15:
        print(f"      ✅ Low risk - feasible to implement")
    elif optimal['prob_overbook'] < 0.30:
        print(f"      ⚠️  Moderate risk - requires mature operations")
    else:
        print(f"      ⛔ High risk - not recommended for implementation")

# Combined Portfolio
print("\n" + "=" * 70)
print("💰 COMBINED PORTFOLIO IMPACT")
print("=" * 70)

# Calculate totals for each phase
hotels = list(simulation_results.keys())

phase1_gains = {}
phase2_gains = {}
phase3_gains = {}
optimal_gains = {}

for hotel_name, data in simulation_results.items():
    results = data['results_df']
    baseline = data['baseline']['avg_revenue']
    
    phase1_gains[hotel_name] = (results[results['overbook_pct'] == 10].iloc[0]['avg_revenue'] - baseline) * 365
    phase2_gains[hotel_name] = (results[results['overbook_pct'] == 16].iloc[0]['avg_revenue'] - baseline) * 365
    phase3_gains[hotel_name] = (results[results['overbook_pct'] == 20].iloc[0]['avg_revenue'] - baseline) * 365
    optimal_gains[hotel_name] = (data['optimal']['avg_revenue'] - baseline) * 365

total_p1 = sum(phase1_gains.values())
total_p2 = sum(phase2_gains.values())
total_p3 = sum(phase3_gains.values())
total_opt = sum(optimal_gains.values())

print(f"\n   Phase 1 (10% overbook):  €{total_p1:,.0f} annually")
print(f"   Phase 2 (16% overbook):  €{total_p2:,.0f} annually")
print(f"   Phase 3 (20% overbook):  €{total_p3:,.0f} annually")
print(f"   Mathematical Optimal:    €{total_opt:,.0f} annually")

print(f"\n🎯 RECOMMENDED START:")
print(f"   Implement 10% overbooking for both hotels")
print(f"   Expected first-year gain: €{total_p1:,.0f}")
print(f"   Scale up based on operational success")

print("\n✅ Recommendations complete!")


💼 BUSINESS-REALISTIC OVERBOOKING STRATEGY

🔍 Available overbooking levels in simulation:
   Resort Hotel: [np.int64(0), np.int64(2), np.int64(4), np.int64(6), np.int64(8), np.int64(10), np.int64(12), np.int64(14), np.int64(16), np.int64(18), np.int64(20), np.int64(22), np.int64(24), np.int64(26), np.int64(28), np.int64(30), np.int64(32), np.int64(34), np.int64(36), np.int64(38), np.int64(40)]
   City Hotel: [np.int64(0), np.int64(2), np.int64(4), np.int64(6), np.int64(8), np.int64(10), np.int64(12), np.int64(14), np.int64(16), np.int64(18), np.int64(20), np.int64(22), np.int64(24), np.int64(26), np.int64(28), np.int64(30), np.int64(32), np.int64(34), np.int64(36), np.int64(38), np.int64(40)]

📊 SCENARIO ANALYSIS: Multiple Risk Tolerance Levels

🏨 RESORT HOTEL

📊 Characteristics:
   Capacity: 96 rooms
   Cancellation Rate: 28.3%
   ADR: €91.07

📈 Overbooking Strategy Options:
       Strategy Overbook Bookings Daily Gain Annual Gain  Risk Incidents/Yr
   Conservative      10%      105  